# Tools

In [ ]:
from matplotlib import pyplot as plt
import numpy as np, os, glob

from diffpy.Structure import loadStructure
import base64

from diffpy.structure.parsers import getParser
from bem import matter, xscalc, xtaloriprobmodel as xopm

In [ ]:
%matplotlib inline

# Isotropic Model

In [ ]:
# calculate xs
import numpy as np
wavelengths = np.arange(0.05, 5.5, 0.001)
T = 300

## Manual input 

In [ ]:
# create material
atoms = [matter.Atom('Ni', (0,0,0)), matter.Atom('Ni', (0.5, 0.5, 0)),
         matter.Atom('Ni', (0.5,0,0.5)), matter.Atom('Ni', (0, 0.5, 0.5))]
a=3.5238
alpha = 90.
lattice = matter.Lattice(a=a, b=a, c=a, alpha=alpha, beta=alpha, gamma=alpha)
fccNi_1 = matter.Structure(atoms, lattice, sgid=225)

In [ ]:
fccNi_1

In [ ]:
calc = xscalc.XSCalculator(fccNi_1, T, max_diffraction_index=4)

## Using CIF file 

In [ ]:
cif_file = "/Users/j35/git/NEUIT/static/cif/ni.cif"
#cif_file = "/Users/j35/git/NEUIT/static/cif/Cu.cif"
cif_file = "/Users/j35/git/NEUIT/static/cif/EntryWithCollCode9805.cif"

### reading manually 

In [ ]:
with open(cif_file, 'r') as f:
    cif_file_content = f.read()

def parse_cif_upload(content):
    cif_s = content
    p = getParser('cif')
    struc = p.parse(cif_s)
    struc.sg = p.spacegroup
    return struc

fcc_2a = parse_cif_upload(cif_file_content)

In [ ]:
calc = xscalc.XSCalculator(fcc_2a, T, max_diffraction_index=4)

### reading via loadStructure 

In [ ]:
# fcc_2b = loadStructure(cif_file, fmt='cif')
# p = getParser('cif')
# # fcc_2.sg = p.spacegroup

In [ ]:
# fcc_2b

In [ ]:
# calc = xscalc.XSCalculator(fcc_2b, T, max_diffraction_index=4)

## Calculation 

In [ ]:
xs = calc.xs(wavelengths)

In [ ]:
plt.plot(wavelengths, xs)

In [ ]:
coh_el_xs = [calc.xs_coh_el(l) for l in wavelengths]
inc_el_xs = [calc.xs_inc_el(l) for l in wavelengths]
inel_xs = [calc.xs_inel(l) for l in wavelengths]
abs_xs = np.array([calc.xs_abs(l) for l in wavelengths])
plt.plot(wavelengths, coh_el_xs)
plt.plot(wavelengths, inc_el_xs)
plt.plot(wavelengths, inel_xs)
plt.plot(wavelengths, abs_xs)
plt.plot(wavelengths, abs_xs+coh_el_xs+inc_el_xs+inel_xs)

# With texture

In [ ]:
texture_model = xopm.MarchDollase()
# by default, r for all hkl are 1.
# use the following form to change r
# make sure l>k>h
texture_model.r[(0,0,1)] = 2
# similarly beta can be changed
# texture_model.beta[(0,0,1)] = np.pi/2

calc = xscalc.XSCalculator(fccNi, T, texture_model)

In [ ]:
xs = calc.xs(wavelengths)

In [ ]:
plt.plot(wavelengths, xs)

# With size effect (dynamical diffraction)

In [ ]:
# use keyword "size". unit: meter
calc = xscalc.XSCalculator(fccNi, T, size=10e-6)

In [ ]:
xs = calc.xs(wavelengths)

In [ ]:
plt.plot(wavelengths, xs)